<div style="color:black; background-color:#FFF3E9; border: 1px solid #FFE0C3; border-radius: 10px; margin-bottom:0rem">
    <p style="margin:1rem; padding-left: 1rem; line-height: 2.5;">
        ©️ <b><i>Copyright 2024 @ Author</i></b><br/>
        <i>Author：
            <b>
            <a href="mailto:your_address@email.com">Wang Jianghai 📨 </a>
            </b>
        </i>
        <br/>
        <i>Date：2024-05-12</i><br/>
        <i>License：</a><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">CC BY-NC-SA 4.0</a></i><br/>
    </p>
</div>

>**Algorithm Principle**：
> - [*Computational Materials Science (Molecular Dynamics)* Algorithm Principle](https://bohrium.dp.tech/notebooks/52743861357)

# LAMMPS

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/2536258452e64b43ac27974ab3ae2662/49457829-152b-44b2-b115-1c22eb400be6.gif)

> **Large-scale Atomic Molecular Massively Parallel Simulator**
> 
> 
> 
> **Citation**:
> 
> **LAMMPS - a flexible simulation tool for particle-based materials modeling at the atomic, meso, and continuum scales**, A. P. Thompson, H. M. Aktulga, R. Berger, D. S. Bolintineanu, W. M. Brown, P. S. Crozier, P. J. in 't Veld, A. Kohlmeyer, S. G. Moore, T. D. Nguyen, R. Shan, M. J. Stevens, J. Tranchida, C. Trott, S. J. Plimpton, [*Comp Phys Comm*, **271** (2022) 10817](https://doi.org/10.1016/j.cpc.2021.108171).

- [LAMMPS Website](https://www.lammps.org/)

- [LAMMPS Github](https://github.com/lammps/lammps)

- [LAMMPS Doc](https://docs.lammps.org/)



## [Unit of measurement](https://en.wikipedia.org/wiki/Unit_of_measurement)

### [SI unit](https://en.wikipedia.org/wiki/International_System_of_Units)

| Quantity name             | Unit name | Unit symbol |
|:-------------------------:|:---------:|:-----------:|
| time                      | second    | s           |
| length                    | metre     | m           |
| mass                      | kilogram  | kg          |
| electric current          | ampere    | A           |
| thermodynamic temperature | Kelvin    | K           |
| amount of substance       | mole      | mol         |
| luminous intensity        | candela   | cd          |



### L-J Unit

#### 1. Original Lenard-Jones Potential

$$
V(r)=4\varepsilon\left[(\frac\sigma r)^{12}-(\frac\sigma r)^6\right]
$$

The first term in the brackets is the short-range repulsion due to the Pauli exclusion principle; the second term is the long-range attraction due to London dispersion. Different elements have different L-J potential parameters.



> - [Interatomic Potentials - LAMMPS Tube](https://lammpstube.com/mdpotentials/)
> 
> The L-J potential parameters between different elements can be calculated as follows:
> 
> $$
\sigma_{AB}=\frac{(\sigma_{AA}+\sigma_{BB})}{2}\\
~\\
\varepsilon_{AB}=\sqrt{\varepsilon_{AA}\cdot\varepsilon_{BB}}
$$
> 
> **Ref*：ARKUNDATO, ARTOTO; SU'UD, ZAKI; ABDULLAH, MIKRAJUDDIN; and SUTRISNO, WIDAYANI (2013) "Molecular dynamic simulation on iron corrosion-reduction in high temperature molten lead-bismuth eutectic," [*Turkish Journal of Physics*: Vol. 37: No. 1, Article 14](https://journals.tubitak.gov.tr/physics/vol37/iss1/14/).



#### 2. Reduced Lenard-Jones Potential

$$
V^{\prime}(r)=4[\frac1{r^{\prime12}}-\frac1{r^{\prime6}}]\\
~\\
V^{\prime}(r)=V(r)/\varepsilon\quad r_i^{\prime}=r_i/\sigma
$$

Other unit reduction scales:

- Mass unit：$m_i^{\prime}=m_i/m$

- Length unit：$r_i^{\prime}=r_i/\sigma$

- Energy unit：$V_i^{\prime}=V_i/\varepsilon$

- Time unit：$t_i^{\prime}=t_i/\tau$

### Other Units

| Quantity          | Metal Units        | Real Units            |
|:-----------------:|:------------------:|:---------------------:|
| Length            | Angstroms ($\AA$)  | Angstroms ($\AA$)     |
| Time              | Picoseconds ($ps$) | Femtoseconds ($fs$)   |
| Energy            | $eV$               | $kcal/mole$           |
| Temperature       | Kelvin ($K$)       | Kelvin ($K$)          |
| Pressure          | Bars               | Atm                   |
| Velocity          | $\AA/ps$           | $\AA/fs$              |
| Force             | $eV/\AA$           | $kcal/(mole\cdot\AA)$ |
| Torque            | $eV$               | $kcal/mole$           |
| Dynamic Viscosity | $eV\cdot ps/\AA^3$ | $kcal/(fs\cdot\AA^3)$ |

## Interatomic Potentials
- [*Interatomic Potentials Repository* (nist.gov)](https://www.ctcms.nist.gov/potentials/)

---

# LAMMPS Example

## 1 Diffusion of gas molecules

### 1.1 Single component gas

In [1]:
%%writefile in.single_comp_diffusion

#---------------Initialize Simulation -----------------------#
units lj                # Unit
dimension 2             # Dimensions
boundary p p p          # Boundary conditions
atom_style atomic       # Atomic type

#-------------- Create Atoms Initial Conditions------------- #
lattice hex 1.0                        # Lattice Type
region box block 0 20 0 10 -0.1 0.1    # Space area
create_box 1 box                       # Simulation Box
region 2 block 5 15 0 10 -0.1 0.1
create_atoms 1 region 2                # Creating atoms based on the crystal lattice
mass 1 1.0                             # Mass
velocity all create 2.5 87287          # Velocity

#---------------- Define Interatomic Potential --------------#
pair_style lj/cut 2.5                  # Atomic interaction potential
pair_coeff 1 1 1.0 1.0 2.5             # Pair-potential parameters
neighbor 0.3 bin                       # Neighbor List
neigh_modify every 20 delay 0 check no # Nearest Neighbor Algorithm
fix 1 all nvt temp 0.5 0.5 0.01        # Time step operation
fix 2 all enforce2d

#--------------- Run MD Simulation --------------------------#
dump 1 all custom 100 toEquil.lammpstrj id type x y z vx vy vz    # Output molecular dynamics simulation information
thermo 500                                                        # Output thermodynamic information interval
run 10000                                                         # Run a molecular dynamics simulation for a specified number of steps

Writing in.single_comp_diffusion


In [ ]:
# !lmp -i in.single_comp_diffusion

**Results Visualization**：

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/cb2c4534386e4a5f8b303a1e07f7fbd0/b774c372-18b8-4542-98b1-55bca32072ac.gif)

### 1.2 Multi-component gas

In [ ]:
%%writefile in.multi_comp_diffusion

#---------------Initialize Simulation -----------------------#
units lj
dimension 2
boundary p p p
atom_style atomic
variable t equal 0.5             # Define system temperature

#-------------- Create Atoms Initial Conditions------------- #
lattice sq 1.0
region box block 0 100 0 100 -0.5 0.5
create_box 2 box
create_atoms 1 random 2500 12345 box
create_atoms 2 random 2500 54321 box        # Randomly create atoms
mass 1 1.0
mass 2 1.0

#---------------- Define Interatomic Potential --------------#
pair_style hybrid lj/cut 2.5 soft 5.0
pair_coeff 1 1 lj/cut 1.0 1.0 2.5
pair_coeff 2 2 lj/cut 1.0 1.0 2.5
pair_coeff 1 2 soft 5.0

#--------------- Run MD Simulation --------------------------#
compute eng all pe/atom                                        # Calculate the potential energy of each atom
compute eatoms all reduce sum c_eng                            # Calculate the potential energy of all atoms
thermo_style custom step temp epair etotal press c_eatoms      # Define the output thermodynamic quantity
thermo 1000
dump id all atom 100 dump.lammpstrj
minimize 1e-4 1e-6 1000 10000                                  # Prevent atoms from getting too close
velocity all create $t 87287
fix nvt all nvt temp $t $t 0.01
run 50000

In [ ]:
%%capture
# !lmp -i in.multi_comp_diffusion

*Before running the dynamics evolution, energy minimization is performed to prevent random generation of atoms that are too close together and cause system instability.*

**Result visualization**:

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/cb2c4534386e4a5f8b303a1e07f7fbd0/a4e5fe9c-3fb6-4ca6-8ee2-063df5628fe0.gif)

## 2 Mechanical Properties

### 2.1 Equilibrium Lattice Constant

A lattice is a mathematical representation of a crystal structure, with each lattice point representing a unit cell.



[**Crystal system**](https://en.wikipedia.org/wiki/Crystal_system)

| Crystal system | Required symmetries of the point group                                       | Point groups | Space groups |
|:--------------:|:----------------------------------------------------------------------------:|:------------:|:------------:|
| Triclinic      | None                                                                         | 2            | 2            |
| Monoclinic     | 1 twofold axis of rotation or 1 mirror plane                                 | 3            | 13           |
| Orthorhombic   | 3 twofold axes of rotation or 1 twofold axis of rotation and 2 mirror planes | 3            | 59           |
| Tetragonal     | 1 fourfold axis of rotation                                                  | 7            | 68           |
| Trigonal       | 1 threefold axis of rotation                                                 | 5            | 25           |
| Hexagonal      | 1 sixfold axis of rotation                                                   | 7            | 27           |
| Cubic          | 4 threefold axes of rotation                                                 | 5            | 36           |



The lattice corresponding to the equilibrium lattice constant has the minimum binding energy; the lowest point of the binding energy curve corresponds to the minimum binding energy $E(a_0)$ and the equilibrium lattice constant $a_0$.



#### 2.1.1 FCC-Ar equilibrium lattice constant

##### 2.1.1.1 Single Point Calculation

In [ ]:
%%writefile in.Ar_single_point

units lj
boundary p p p
atom_style atomic
lattice fcc 1.00
region box block 0 1 0 1 0 1
create_box 1 box
create_atoms 1 box
mass 1 1.0
pair_style lj/cut 8.0
pair_coeff 1 1 1.0 1.0 8.0
variable P equal pe
variable L equal (count(all)/1.00)^(1/3)
run 0
print “Lattice constant: $L"
print "Cohesive Energy of Ar: $P"

The L-J potential parameters of Ar are as follows:

|     | $\sigma(nm)$ | $\epsilon(J)$ |
|:---:|:------------:|:-------------:|
| Ar  | 1.3405       | 1.6540 E-21    |

*Ref*: [*Dokl Phys Chem* **472**, 16–18 (2017)](https://doi.org/10.1134/S0012501617010043)

##### 2.1.1.2 Loop Control



In [ ]:
%%writefile in.Ar_loop

units lj
boundary p p p
atom_style atomic

label loop_i
variable i  loop  50
variable x  equal  1.02+0.002*$i

lattice fcc $x
region     box block 0 1 0 1 0 1
create_box 1 box
create_atoms 1 box
mass 1 1.0

pair_style  lj/cut  4.0
pair_coeff  1  1  1.0  1.0  4.0

variable P equal pe
variable L equal (count(all)/$x)^(1/3)

run 0
print  "Cohesive Energy of Ar a = $L E = $P"
clear
next i
jump SELF loop_i

**Data extraction**：

In [ ]:
# Read the file and search for lines containing the pattern "Cohesive Energy of Ar a = * E = *
import re

# Initialize list to hold the matching lines
matching_lines = []

# Define the regex pattern for capturing a and E values
pattern = re.compile(r"Cohesive Energy of Ar a = ([\d\.-]+)\s+E = ([\d\.-]+)")

# Read the file and search for the pattern
with open('./log.lammps', 'r') as file:
    for line in file:
        match = pattern.search(line)
        if match:
            a_value = float(match.group(1))
            E_value = float(match.group(2))
            matching_lines.append((a_value, E_value))

# Write the extracted a and E values to a CSV file
with open('Ar_fcc.csv', 'w') as f:
    f.write("a,E\n")  # Write header
    for a, E in matching_lines:
        f.write(f"{a},{E}\n")

**Results Visualization**：

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


filename = "Ar_fcc.csv"
DataFrame = pd.read_csv(filename)
lat = np.array(DataFrame['a'])
ene = np.array(DataFrame['E'])

a, b, c = np.polyfit(lat, ene, 2)
mesh = np.linspace(np.min(lat), np.max(lat), 1000)
opt_lat = -b / (2 * a)
Emin = np.polyval([a, b, c], opt_lat)
print("a,b,c =", a, b, c)
print("Emin =", Emin, "opt_lat =", opt_lat)

plt.scatter(lat, ene, 10)
plt.plot(mesh, a * mesh ** 2 + b * mesh + c)
plt.xlabel('Lattice parameter')
plt.ylabel('Energy')
plt.show()

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/cc022afdb749478486df31dfedadb978/bd1a856d-d9e2-4617-9a09-dc978643ad43.png)



- Theoretical equilibrium lattice constant：

$$
\frac{\partial u}{\partial r}=-2\varepsilon[nA_{n}(\frac{\sigma^{n}}{r^{n+1}})-mA_{m}(\frac{\sigma^{m}}{r^{m+1}})]\\
~\\
-2\varepsilon[nA_{n}(\frac{\sigma^{n}}{r^{n+1}})-mA_{m}(\frac{\sigma^{m}}{r^{m+1}})]=0\\
~\\
r_{0}=(\frac{2A_{12}}{A_{6}})^{1/6}\sigma=1.09\sigma\\
~\\
\text{a}=\sqrt{2}r_0=1.54\sigma
$$

#### 2.1.2 FCC-Al equilibrium lattice constant

##### 2.1.2.1 Loop Control

In [ ]:
%%writefile in.Al_loop

units metal
boundary p p p
atom_style atomic
variable i loop 30
variable x equal 3.90+0.01*$i

# Define a lattice for use by other commands.
lattice fcc $x
region box block 0 1 0 1 0 1
create_box 1 box
create_atoms 1 box

pair_style eam/fs
pair_coeff * * Al_mm.eam.fs Al

variable n equal count(all)
variable P equal pe/$n

run 0
print "Cohesive Energy of Al a = $x E = $P"
clear 
next i
jump SELF

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/8f49555cc5cc4477ba30f8a8a44d6639/fe7abbc3-81f8-4cf1-97d7-ea9956a0ef43.png)

##### 2.1.2.2 Lattice Relaxation

Crystal structure optimization using the conjugate gradient method:

In [ ]:
%%writefile in.Al_relax

units metal
dimension 3
boundary p p p
atom_style atomic

lattice fcc 4.00
region box block 0 1 0 1 0 1 units lattice
create_box 1 box
create_atoms 1 box

pair_style eam/fs
pair_coeff * * Al_mm.eam.fs Al

neighbor 2.0 bin
neigh_modify delay 10 check yes

compute eng all pe/atom
compute eatoms all reduce sum c_eng
dump 1 all atom 1 relax.lammpstrj

#--------------Run minimization--------------#
reset_timestep 0
fix 1 all box/relax iso 0.0 vmax 0.001
thermo 10
thermo_style custom step pe lx ly lz press pxx pyy pzz c_eatoms

min_style cg
minimize 1e-25 1e-25 5000 10000

variable natoms equal count(all)
variable teng equal "c_eatoms"
variable a equal lx
variable ecoh equal "v_teng/v_natoms“

print "Total energy (eV)= ${teng};" 
print "Number of atoms = ${natoms};"
print "Lattice constant (Angstroms) = ${a};" 
print "Cohesive energy (eV/atom) = ${ecoh};" 

#### 2.1.3 Notes

- A too **small** lattice constant fitting range will amplify the **truncation error**, affecting calculation accuracy.

- A too **large** lattice constant fitting range will produce **anharmonic effects**, causing the actual curve to deviate from the quadratic form.

By enumerating common lattice types and fitting the lowest energy corresponding to the equilibrium lattice constant, the most stable configuration of an element can be determined.

### 2.2 Bulk modulus

[Bulk modulus](https://en.wikipedia.org/wiki/Bulk_modulus) is a measure of the **compressibility** of a substance.

#### 2.2.1 Calculation Outline

When $T=0$, the pressure can be expressed as

$$
P=-\frac{\partial U}{\partial V}
$$

Where $U$ is the total energy of the system. Define the energy and volume of a unit atom:

$$
u=\frac UN\quad v=\frac VN
$$

then

$$
\begin{aligned}B&=-V\left(\frac{\partial P}{\partial V}\right)_T\\&=V\frac{\partial^2U}{\partial V^2}\\&=v\frac{\partial^2u}{\partial v^2}\end{aligned}
$$

#### 2.2.2 Parsing Expression

$$
\begin{aligned}\text{u}&=2\varepsilon[\sum_{i\neq j}(\frac{1}{M_{ij}})^{12}(\frac{\sigma}{r})^{12}-\sum_{i\neq j}(\frac{1}{M_{ij}})^{6}(\frac{\sigma}{r})^{6}]\\&=2\varepsilon[A_{12}\left(\frac\sigma r\right)^{12}-A_6\left(\frac\sigma r\right)^6]\end{aligned}
$$

where

$$
A_{12}=\sum_{j}\frac{1}{M_{ij}^{12}}=12.13\\A_{6}=\sum_{j}\frac{1}{M_{ij}^{6}}=14.45
$$

thus

$$
\begin{aligned}B&=v\frac{\partial^2u}{\partial v^2}\\&=\frac v{\left(\frac{\partial v}{\partial r}\right)^2}\cdot\frac{\partial^2u}{\partial r^2}\\&=\frac{\sqrt{2}}{9r}\cdot\frac{\partial^2u}{\partial r^2}|_{r=r_0}\end{aligned}
$$

> For **FCC crystal**,
> 
> $$
\begin{cases} v=\frac{a^3}4=\frac{r^3}{\sqrt{2}}\\\\ \left(\frac{\partial v}{\partial r}\right)^2=\frac{9r^4}2\end{cases}
$$

$$
\begin{aligned}
u&=2\varepsilon[A_{12}\left(\frac{\sigma}{r}\right)^{12}-A_6\left(\frac{\sigma}{r}\right)^6] \\\\
\frac{\partial u}{\partial r}&=2\varepsilon[\frac{(-12)A_{12}\sigma^{12}}{r^{13}}-\frac{(-6)A_{6}\sigma^{6}}{r^{7}}] \\\\
\frac{\partial^2u}{\partial r^2}&=2\varepsilon[\frac{(-13)(-12)A_{12}\sigma^{12}}{r^{14}}-\frac{(-7)(-6)A_6\sigma^6}{r^8}] \\
\end{aligned}
$$

Substitute

$$
\begin{cases} r_0=1.09\sigma\\\\ \frac{\partial^2u}{\partial r^2}|_{r_0}=523\frac\varepsilon{\sigma^2}\end{cases}
$$

into

$$
B=\frac{\sqrt{2}}{9r_0}\cdot\frac{\partial^2u}{\partial r^2}|_{r=r_0}
$$

we get the theoretical bulk modulus

$$
B_0=75\frac\varepsilon{\sigma^3}
$$

#### 2.2.3 L-J potential bulk modulus

- **Formula:**: $B=v\frac{\partial^2u}{\partial v^2}$

- **Method**: Continuously adjusted the lattice constant and used LAMMPS to calculate the relationship between the Cu crystal energy and its volume near the equilibrium distance to **fit** the bulk modulus data.

In [ ]:
%%writefile in.fcc_bulk_modulus

units lj
boundary p p p
atom_style atomic
label LOOP

# The volume change is controlled by lattice constant
variable i loop 40
variable x equal 1.05+0.002*$i

lattice fcc $x
region box block 0 1 0 1 0 1
create_box 1 box
create_atoms 1 box

pair_style lj/cut 8.50
pair_coeff * * 1.0 1.0
mass 1 1.0

thermo_style custom etotal
variable P equal pe                  # Potential energy per atom
variable v equal (1.0/$x)            # vol in LJ unit is N/x, vol per atom in LJ units is 1/x

dump 1 all custom 10 trj.lammpstrj id type x y z vx vy vz
run 0
print "FCC lattice rho = $v E = $P"
clear
next i
jump SELF LOOP

Data processing:

In [ ]:
# Read the file and search for lines containing the pattern "FCC lattice rho = * E = *"
import re

# Initialize list to hold the matching lines
matching_lines = []

# Define the regex pattern for capturing a and E values
pattern = re.compile(r"FCC lattice rho = ([\d\.-]+)\s+E = ([\d\.-]+)")

# Read the file and search for the pattern
with open('./log.lammps', 'r') as file:
    for line in file:
        match = pattern.search(line)
        if match:
            v_value = float(match.group(1))
            E_value = float(match.group(2))
            matching_lines.append((v_value, E_value))

# Write the extracted a and E values to a CSV file
with open('fcc_bulk_modulus.csv', 'w') as f:
    f.write("V,E\n")  # Write header
    for a, E in matching_lines:
        f.write(f"{V},{E}\n")

##### 2.2.3.1 Quadratic Function Fitting

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


filename = "fcc_bulk_modulus.csv"
DataFrame = pd.read_csv(filename)
vol = np.array(DataFrame['V'])
ene = np.array(DataFrame['E'])

a, b, c = np.polyfit(vol, ene, 2)
mesh = np.linspace(np.min(vol), np.max(vol), 1000)
opt_vol = -b / (2 * a)
Emin = np.polyval([a, b, c], opt_vol)
print("a,b,c =", a, b, c)
print("Emin =", Emin, "opt_vol =", opt_vol)

plt.scatter(vol, ene, 10)
plt.plot(mesh, a * mesh ** 2 + b * mesh + c)
plt.xlabel('Volume')
plt.ylabel('Energy')

plt.show()

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/cc022afdb749478486df31dfedadb978/15e2ee08-e5db-45fe-90c0-edfe9c8eff4e.png)

> There is a certain **anharmonic effect** in the E-V data.

##### 2.2.3.2 [Birch-Murnaghan equation of state](https://en.wikipedia.org/wiki/Birch–Murnaghan_equation_of_state) Fitting

$$
E\left(V\right)=E_0+\frac{B_0V}{B_0^{\prime}}{\left(\frac{(V_0/V)^{B_0^{\prime}}}{B_0^{\prime}-1}+1\right)-\frac{B_0V_0}{B_0^{\prime}-1}}
$$

The LAMMPS calculation results are $E$, $V$, and the parameters to be fitted are $E_0$, $V_0$, $B_0$, and $B_0^{\prime}$.

In [ ]:
import pandas as pd
import numpy as np


df = pd.read_csv('fcc_bulk_modulus.csv')
v = np.array(df['V'])
e = np.array(df['E'])

a, b, c = np.polyfit(v, e, 2)
v0 = -b / (2 * a)
e0 = a * v0 ** 2 + b * v0 + c
b0 = 2 * a * v0
bP = 3.5
x0 = [e0, b0, bP, v0]


# Birch–Murnaghan equation of state
def Murnaghan(parameters, vol):
    E0 = parameters[0]
    B0 = parameters[1]
    BP = parameters[2]
    V0 = parameters[3]
    E = E0 + B0 * vol / BP * (((V0 / vol) ** BP) / (BP - 1) + 1) - V0 * B0 / (BP - 1.)

    return E


# error
def residual(pars, y, x):
    err = y - Murnaghan(pars, x)

    return err


from scipy.optimize import leastsq

murnpars, ier = leastsq(residual, x0, args=(e, v))

print('Bulk Modulus:' + str(murnpars[1]))
print('lattice constant:', murnpars[3] ** (1 / 3))

from matplotlib import pyplot as plt

v_mesh = np.linspace(np.min(v), np.max(v), 1000)
plt.scatter(v, e, 10)
plt.plot(v_mesh, Murnaghan(murnpars, v_mesh))
plt.xlabel('Volume')
plt.ylabel('Energy')

plt.show()

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/cc022afdb749478486df31dfedadb978/d02d4341-e97a-4753-a8c4-8106134553df.png)

> **The Birch-Murnaghan equation of state is closer to physical reality** and the results obtained are closest to the analytical solution. Therefore, **the Birch-Murnaghan equation of state should be preferred for fitting the bulk modulus.**

#### 2.2.4 Cu bulk modulus

The L-J potential parameters of Cu are as follows:


|     | $\sigma(nm)$ | $\epsilon(J)$ |
|:---:|:------------:|:-------------:|
| Cu  | 0.2338       | 65.5815 E-21  |


*Ref*: [*Nanoscale Res Lett* **6**, 200 (2011)](https://doi.org/10.1186/1556-276X-6-200)

##### 2.2.4.1 L-J potential

In [ ]:
%%writefile in.Cu_lj_mudulus

units metal
boundary p p p
atom_style atomic
#------------- setup loop -------------#
variable i loop 40
variable x equal 3.40+0.01*$i
lattice fcc $x
region box block 0 1 0 1 0 1
create_box 1 box
create_atoms 1 box
mass 1 64

#------------- Cu LJ parameter -------------#
pair_style lj/cut 10.0
pair_coeff 1 1 0.40933 2.338
variable v equal ($x)^3
variable n equal count(all)
variable P equal pe

#------------- run -------------#
run 0
print "Cohesive Energy of Cu v = $v x = $x E = $P "
clear 
next i
jump SELF

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/8f49555cc5cc4477ba30f8a8a44d6639/d643285f-d050-4a30-b4d8-d815a6781e76.png)

> **The L-J potential function cannot accurately describe the interaction between metal atoms!**

##### 2.2.4.2 EAM Potential

In [ ]:
%%writefile in.Cu_eam_mudulus

units metal
boundary p p p
atom_style atomic
#------------- setup loop -------------#
variable i loop 40
variable x equal 3.40+0.01*$i
lattice fcc $x
region box block 0 1 0 1 0 1
create_box 1 box
create_atoms 1 box
mass 1 64

#------------- Cu EAM parameter -------------#
pair_style eam
pair_coeff * * Cu_u3.eam
variable v equal ($x)^3
variable n equal count(all)
variable P equal pe

#------------- run -------------#
run 0
print "Cohesive Energy of Cu v = $v x = $x E = $P "
clear 
next i
jump SELF

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/8f49555cc5cc4477ba30f8a8a44d6639/a11730cb-202b-47b5-a734-694d47d31d5b.png)

### 2.3 Tensile properties

#### 2.3.1 Stress-strain curve

The relationship between [**stress**](https://en.wikipedia.org/wiki/Stress_(mechanics)) and [**strain**](https://en.wikipedia.org/wiki/Strain_(mechanics)) is called the **stress-strain curve** of a material.

$$
\begin{aligned}\sigma&=\frac F{A_0}\\\\\epsilon&=\frac{L-L_0}{L_0}=\frac{\Delta L}{L_0}\end{aligned}
$$

**Graphene tensile test**：
- Graphene modeling

In [ ]:
%%writefile in.graphene

# Parameters
variable        Lx      equal   20
variable        Ly      equal   10
# variable      Lz      equal   0.8

# Unit Size
variable        x         equal    ${Lx}
variable        y         equal    ${Ly}
# variable        z         equal    ${Lz}
variable        xbox         equal    round(v_x)
variable        ybox         equal    round(v_y)
# variable        zbox         equal    round(v_z)

# Initialization
units                real
dimension            3
boundary             p p p
neighbor             2.0 bin
neigh_modify         every 10 delay 0 check yes
timestep             0.001
atom_style           charge

# Modeling
lattice custom  1.421 a1  3  0  0   a2   0 1.732  0 a3 0 0 2.357   &
        basis   0        0  0  &
        basis   0.333    0  0  &
        basis   0.5     0.5 0  &
        basis   0.833   0.5 0
region                box block 0 ${xbox} 0 ${ybox} -5.0 5.0
create_box            1 box
region                graphene block 0 ${xbox} 0 ${ybox} -0.1 0.1
create_atoms          1 region graphene
mass                  * 12.011150
write_data            graphene.dat

- Graphene stretching

In [ ]:
%%writefile in.tensile

# UNITS
units real
timestep 1
variable fpress equal 0.000101325        # atm -> GPa
variable fenergy equal 0.043             # kcal/mole -> eV

# GRAPHENE SHEET
dimension 3
boundary p p f
atom_style charge
read_data graphene.dat
pair_style reax/c NULL checkqeq no
pair_coeff * * ffield.reax.cho C

# TEMPERATURE SETTINGS
variable temp equal 300
variable seed equal 1717
velocity all create ${temp} ${seed} dist uniform

# EQUILIBRATION
fix fnpt all npt temp ${temp} ${temp} 10 x 0 0 500 y 0 0 500
thermo 100
run 1000

# OUTPUT
# assume that the thickness of monolayer graphene is 3.35A.
variable tmp equal lx
variable lx0 equal ${tmp}
variable tmp equal ly
variable ly0 equal ${tmp}
variable Eavg equal etotal/atoms*${fenergy}                # eV/atom
variable pe equal pe/atoms*${fenergy}                      # eV/atom
variable ke equal ke/atoms*${fenergy}                      # eV/atom
variable strainx equal (lx-${lx0})/${lx0}
variable strainy equal (ly-${ly0})/${ly0}
variable stressx equal -pxx*(lz/3.35)*${fpress}            # GPa
variable stressy equal -pyy*(lz/3.35)*${fpress}            # GPa
thermo_style custom step time temp etotal press v_Eavg v_pe v_ke v_strainx v_stressx v_strainy v_stressy

# DEFORMATION
fix boxdeform all deform 1 x scale 2 remap x
fix fnpt all npt temp ${temp} ${temp} 10 y 0 0 500
fix output all ave/time 1 100 100 v_strainx v_stressx v_strainy v_stressy file stress_strain.txt
dump 1 all atom 10 dump.lammpstrj
fix stop all halt 100 v_strainx > 0.7  error  continue
thermo 100
run 10000

*Graphene stretching demonstration*:

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/9eea4c3ad7fd4a9e9a8e2ca50b8c4fad/7cb4df51-0e93-4b89-96ce-22d356209071.gif)

*Stress-strain curve*:
![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/58f11699c79f4f4db5b19d22961bed65/a3d71a42-21a4-4fe5-bf37-7d582f7f0f2a.png)

#### 2.3.2 Elastic constant

The [**Elastic constant**](https://en.wikipedia.org/wiki/Elastic_modulus)characterizes the elasticity of a material. During elastic deformation, stress and strain obey [*Hooke's law*](https://en.wikipedia.org/wiki/Hooke%27s_law). The generalized elastic tensor relating stress and strain in anisotropic media has **81** components, **21** of which are independent and **can be simplified based on crystal symmetry**.

$$
\sigma_{ij}=C_{ijkl}\varepsilon_{kl}\\
~\\
c_{ijmn}=c_{jimn}=c_{ijnm}=c_{jinm}=c_{mnij}=c_{nmij}=c_{mnji}=c_{nmji}\\
~\\
\left.\left[\begin{array}{ccccccc}c_{1111}&c_{1122}&c_{1133}&c_{1123}&c_{1113}&c_{1112}\\&c_{2222}&c_{2233}&c_{2223}&c_{2213}&c_{2212}\\&&c_{3333}&c_{3323}&c_{3313}&c_{3312}\\&&&c_{2323}&c_{2313}&c_{2312}\\&&&&c_{1313}&c_{1312}\\&&&&&c_{1212}\end{array}\right.\right]
$$


Taking FCC-Cu as an example, according to the relationship between symmetry and the number of independent elastic constants, materials with cubic symmetry have three independent elastic constants:

$$
\left.C=\left(\begin{array}{cccccc}C_{11}&C_{12}&C_{12}&0&0&0\\C_{12}&C_{11}&C_{12}&0&0&0\\C_{12}&C_{12}&C_{11}&0&0&0\\0&0&0&C_{44}&0&0\\0&0&0&0&C_{44}&0\\0&0&0&0&0&C_{44}\end{array}\right.\right)
$$

Elastic internal energy of the system:

$$
E^\text{elas}/V=\frac12\sum_{ij}C_{ij}\varepsilon_i\varepsilon_j
$$

Therefore, the corresponding elastic deformation can be constructed to calculate the elastic constant.



##### 2.3.2.1 Stiffness tensor

$$
C=\left(\begin{array}{cccccc}C_{11}&C_{12}&C_{12}&0&0&0\\C_{12}&C_{11}&C_{12}&0&0&0\\C_{12}&C_{12}&C_{11}&0&0&0\\0&0&0&C_{44}&0&0\\0&0&0&0&C_{44}&0\\0&0&0&0&0&C_{44}\end{array}\right)
$$


##### 2.3.2.2 Flexibility Tensor

$$
[S]=[C]^{-1}
$$


$$
\left.S=\left(\begin{array}{cccccc}\frac{C_{11}+C_{11}}{C_{11}^2+C_{11}C_{12}-2C_{12}^2}&\frac{-C_{12}}{C_{11}^2+C_{11}C_{12}-2C_{12}^2}&\frac{-C_{12}}{C_{11}^2+C_{11}C_{12}-2C_{12}^2}&0&0&0\\\frac{-C_{12}}{C_{11}^2+C_{11}C_{12}-2C_{12}^2}&\frac{C_{11}+C_{11}C_{12}-2C_{12}^2}{C_{11}^2+C_{11}C_{12}-2C_{12}^2}&\frac{-C_{12}}{C_{11}^2+C_{11}C_{12}-2C_{12}^2}&0&0&0\\\frac{-C_{12}}{C_{11}^2+C_{11}C_{12}-2C_{12}^2}&\frac{-C_{11}}{C_{11}^2+C_{11}C_{12}-2C_{12}^2}&\frac{C_{11}+C_{12}}{C_{11}^2+C_{11}C_{12}-2C_{12}^2}&0&0&0\\0&0&0&1/C_{44}&0&0\\0&0&0&0&0&1/C_{44}\end{array}\right.\right)
$$


#### 2.3.3 Young's modulus

When an elastic material is subjected to normal stress, it will produce normal strain. **The ratio of normal stress to normal strain** is defined as the material's[**Young's modulus**](https://en.wikipedia.org/wiki/Young%27s_modulus):

$$
Y=\frac\sigma\varepsilon=\frac{F/_{A_0}}{\Delta L/_{L_0}}=\frac{FL_0}{A_0\Delta L}
$$

When an elastic material is subjected to uniaxial tension, the only stress is $\sigma_{xx}$

$$
\varepsilon_{xx}=S_{11}\sigma_{xx}\\~\\
Y=\frac{\sigma_{xx}}{\varepsilon_{xx}}=\frac1{S_{11}}=\frac{C_{11}{}^2+C_{11}C_{12}-2C_{12}{}^2}{C_{11}+C_{12}}
$$

Where $S_{11}$ is the $(1,1)$ component of the flexibility tensor.



#### 2.3.4 Poisson's ratio

**The ratio of the amount of deformation in the lateral direction to the amount of deformation in the longitudinal direction** when a material is stretched or compressed is called [**Poisson's ratio**](https://en.wikipedia.org/wiki/Poisson%27s_ratio).

$$
v=-\frac{\varepsilon_{\mathrm{lateral}}}{\varepsilon_{\mathrm{axial}}}
$$



$$
\varepsilon_{xx}=S_{11}\sigma_{xx}\\\varepsilon_{yy}=S_{21}\sigma_{xx}\\~\\\nu=\frac{\varepsilon_{yy}}{\varepsilon_{xx}}=\frac{S_{21}}{S_{11}}=\frac{-C_{12}}{C_{11}+C_{12}}
$$

## 3 Thermodynamic Properties

### 3.1 Thermodynamic Basics

#### 3.1.1 [Fundamental laws of thermodynamics](https://en.wikipedia.org/wiki/Laws_of_thermodynamics)

- [**Zeroth law of thermodynamics**](https://en.wikipedia.org/wiki/Zeroth_law_of_thermodynamics) - Law of thermal equilibrium
  
  > If two thermodynamic systems are each in thermal equilibrium (at the same temperature) with a third thermodynamic system, then they must also be in thermal equilibrium with each other.
  > 
  > All substances in thermodynamic equilibrium have **a certain common macroscopic physical property**.
  
  The zeroth law of thermodynamics determines the state function - **Temperature**.

- [**First Law of Thermodynamics**](https://en.wikipedia.org/wiki/First_law_of_thermodynamics) - Law of Conservation of Energy
  
  > $$
  \Delta U=Q+W
  $$
  > 
  > The increase in the internal energy of an object is equal to the sum of the heat absorbed by the object and the work done on the object.
  > 
  > [The First kind of perpetual motion machine](https://en.wikipedia.org/wiki/Perpetual_motion#Classification) is impossible to realize.
  
  The first law of thermodynamics determines the state functions - *Internal energy* and *Enthalpy*.

- [**Second Law of Thermodynamics**](https://en.wikipedia.org/wiki/Second_law_of_thermodynamics) - Principle of Entropy Increase
  
  > $$
  \eta=\frac A{Q_1}=1-\frac{Q_2}{Q_1}<1
  $$
  > 
  > Clausius's statement: Heat cannot spontaneously transfer from a cold object to a hot object.
  > 
  > Kelvin's statement: It is impossible to extract heat from a single heat source and convert it completely into useful work without causing other effects.
  > 
  > Principle of entropy increase: The entropy of an isolated system never decreases automatically. The entropy remains unchanged in a reversible process and increases in an irreversible process.
  > 
  > [The Second kind of perpetual motion machine](https://en.wikipedia.org/wiki/Perpetual_motion#Classification) is impossible to realize.

- [**Third law of thermodynamics**](https://en.wikipedia.org/wiki/Third_law_of_thermodynamics) - Nernst Theorem
  
  > $$
  \lim_{T\to0K}\left(\Delta S\right)_{T}=0
  $$
  > 
  > Planck's statement: As the temperature approaches 0 $K$, the entropy of all *perfect crystals* approaches a universal constant, defined as 0.
  > 
  > Nernst's statement: If the temperature of an isothermal reversible process is close to $0K$, the entropy change associated with any condensed system undergoing the process is close to zero.
  > 
  > **Unreachability Principle**: No process, no matter how idealized, can reduce the entropy of a system to its absolute zero value within a finite number of operations.



#### 3.1.2 Microscopic thermal motion

According to the principle that **absolute zero is unattainable**, atoms possess velocity and kinetic energy at any finite temperature.

- Solids: lattice vibrations;

- Liquids: Brownian motion;

- Gases: molecular collisions.

For crystals, lattice vibrations are a typical form of thermal motion and contribute primarily to their thermal properties, such as heat capacity, thermal expansion, melting, sintering, and heat conduction.



#### 3.1.3 Thermal expansion and contraction

Thermal expansion and contraction is a property of most objects. Under normal conditions, objects will expand when heated and shrink when cooled.

> - Generally speaking, when the temperature rises, the kinetic energy of the molecules increases and the mean free path increases, which manifests as thermal expansion; when the temperature drops, the kinetic energy of the molecules decreases and the mean free path decreases, which manifests as contraction.
> 
> - Water is an exception. **The hydrogen bonds between water molecules are directional.** Within a certain temperature range, as the temperature drops, the number of hydrogen bonds in water increases, causing the volume to increase as the temperature drops.



#### 3.1.4 Thermodynamic properties

Specific Heat Capacity

Specific heat capacity is a physical quantity that measures **a substance's ability to hold heat**. It is the energy required to raise the temperature of an object per unit mass or volume by a certain amount.

Microscopic Explanation: The temperature of an object is often measured by the intensity of the motion of microscopic particles. Factors that affect the velocity of these particles can affect the object's specific heat.

- Solids have strong interparticle interactions. The movement of one atom easily affects the movement of other atoms, leading to an overall temperature increase and a relatively low specific heat.

- Liquids have weaker interparticle interactions than solids. The movement of a particle requires more energy to affect other particles, resulting in a relatively high specific heat.

Physical State

As temperature rises, materials generally undergo a phase transition **from solid to liquid to gas**.



### 3.2 Coefficient of thermal expansion

[**Thermal expansion**](https://en.wikipedia.org/wiki/Thermal_expansion) is the tendency of a substance to change its shape, area, volume, and density in response to changes in temperature. The degree to which a material expands or contracts when heated or cooled can be measured using its coefficient of thermal expansion.

The coefficient of thermal expansion is generally defined in three ways:

- Linear expansion coefficient:
  
  $$
  \alpha_L=\frac{1}{L}\frac{\mathrm{d}L}{\mathrm{d}T}
  $$

- Coefficient of face expansion:
  
  $$
  \alpha_A=\frac{1}{A} \frac{\mathrm{d}A}{\mathrm{d}T}
  $$

- Coefficient of volume expansion:
  
  $$
  \alpha_V=\frac{1}{V}\frac{\mathrm{d}V}{\mathrm{d}T}
  $$

For isotropic materials,

$$
\alpha_A=2\alpha_L\\~\\\alpha_V=3\alpha_L
$$



#### 3.2.1 Calculation Outline

Using the *NPT* ensemble, calculate the volume of $Cu$ at different temperatures. The slope of the curve is $\Delta V/\Delta T$.

#### 3.2.2 Input File

In [ ]:
%%writefile in.thermal_expansion_coefficent

variable T index 300 400 500 600 700 800 900 1000
label T_loop

units metal
boundary p p p
atom_style atomic

# Create lattice
lattice fcc 3.62
region box block 0 8 0 8 0 8
create_box 1 box
create_atoms 1 box

# Set interatomic potential
pair_style eam
pair_coeff 1 1 Cu_u3.eam

# Reset timestep
reset_timestep 0

# Initialize velocity
velocity all create ${T} 87287 dist gaussian

# Equilibrate using NPT ensemble
fix 1 all npt temp ${T} ${T} $(100*dt) iso 0 0 1

# Define thermo output
thermo_style custom step temp epair press lx ly lz
thermo 1000

# Define temperature, length and volume computation
compute actual_T all temp
variable Lx equal lx
variable V equal vol

# Average properties over time and write to a single file
fix 2 all ave/time 100 10 10000 c_actual_T v_Lx v_V file thermal_expansion_data.${T}.txt

# Run simulation
run 10000

# Unfix the NPT ensemble
unfix 1

clear
next T
jump SELF T_loop

*Fitting data*:

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/518c77d1e3d147e597b76d200bd4e3ea/27437a7b-fc82-41ec-8f20-3cdfa1b0cc63.png)

$$
\frac{\Delta L}{\Delta T} = 5.55 \times 10^{-4} \AA / K
$$

### 3.3 Specific heat capacity

[**Specific heat capacity**](https://en.wikipedia.org/wiki/Specific_heat_capacity): The amount of heat absorbed or released per unit mass or volume of a substance when its temperature increases or decreases by $1K$.

- Volumetric heat capacity:
  
  $$
  C_{V}=\frac{\Delta E}{\Delta T\cdot V}
  $$

- Mass heat capacity:
  
  $$
  C_{V}=\frac{\Delta E}{\Delta T\cdot V\cdot \rho}
  $$

#### 3.3.1 Calculation Outline

Using the *NVT* ensemble, the volume is kept constant during the simulation and the changes in system energy with temperature are considered.

$$
C_V=\left(\frac{dE}{dT}\right)_V
$$



#### 3.3.2 Input File

In [ ]:
%%writefile in.Cv

units metal 
boundary p p p
atom_style atomic

variable Tstart equal 10.0
variable Tstop equal 1000
variable Tdamp equal 0.2

lattice fcc 3.62
region simbox block 0 8 0 8 0 8
create_box 1 simbox
create_atoms 1 box

pair_style eam
pair_coeff 1 1 Cu_u3.eam

velocity all create ${Tstart} 825577 dist gaussian

# Equilibrium step
fix 1 all nvt temp ${Tstart} ${Tstart} ${Tdamp}
thermo 100
run 5000
unfix 1
reset_timestep 0
thermo 1000
thermo_style custom step temp pe etotal

# Heating step
print "Production steps started!"
fix 2 all nvt temp ${Tstart} ${Tstop} ${Tdamp}
run 120000 
print "Simulation complete!"

*Fitting data*：

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/518c77d1e3d147e597b76d200bd4e3ea/9ce257fd-7e61-4f3e-8138-c4c242a9685d.png)

$$
\frac{\Delta E}{\Delta T} = 0.528 eV/K
$$

### 3.4 phase transition

The process of a crystal changing from solid to liquid is called **melting**, and the corresponding temperature is the **melting point**; the process of a crystal changing from liquid to solid is called **solidification**, and the corresponding temperature is the **freezing point**.

The melting and solidification of crystals are typical [**first-order phase transitions**](https://en.wikipedia.org/wiki/Phase_transition#Modern_classifications). During the phase transition, the chemical potential of the two phases is continuous, but the first-order derivative of the chemical potential is discontinuous.

$$
\mu_1(T,p)=\mu_2(T,p)\\~\\\frac{\partial\mu_1}{\partial T}\neq\frac{\partial\mu_2}{\partial T},\frac{\partial\mu_1}{\partial p}\neq\frac{\partial\mu_2}{\partial p}
$$

According to the definition of $Gibbs$ free energy, there will be a sudden change in the *entropy* and *volume* of the system, accompanied by the occurrence of latent heat of phase change.

#### 3.4.1 Calculation Outline

Using the **NPT ensemble**, phase transitions can be identified by monitoring **changes in volume**, **enthalpy**, or **other parameters representing order (entropy)**, thereby determining the melting or freezing point of a crystal.

Specifically, the following **transition indices** are used:

- Volume-temperature change ($V - T$)

- Mean square displacement (MSD)

- Radial distribution function (RDF)

- Lindemann index (*Lindemann index*)



3.4.2 Melting and Solidification

3.4.2.1 Melting Process

In [ ]:
%%writefile in.fusion

units lj
boundary p p p
atom_style atomic

lattice fcc 1.073
region box block 0 10 0 10 0 10
create_box 1 box
create_atoms 1 box
mass 1 1.0

pair_style lj/cut 5.0
pair_coeff 1 1 1.0 1.0 5.0
velocity all create 0.01 87287

timestep 0.005

# Equilibrium step
fix 1 all nvt temp 0.01 0.01 0.2
run 50000
unfix 1

# Heating step
reset_timestep 0
thermo 1000
thermo_style custom step temp pe etotal press vol
fix 1 all npt temp 0.01 0.85 1.0 iso 0 0 1.0
dump 1 all atom 10000 fusion.lammpstrj
run 1000000

*Melting process demonstration*：

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/da67be899d33469ba0405ab9411d9c77/ff392bf3-063b-4323-9061-761836485c20.gif)

*V-T diagram*：

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/dbe40ddd44a548ecb276094754114c6b/1dd8f53e-c0bd-4412-a97a-ed59ac0eab1d.png)

The calculation results show that the molecular dynamics simulation of the melting process will produce **overheating** phenomenon.

##### 3.4.2.2 Solidification process


In [ ]:
%%writefile in.quench

units lj
boundary p p p
atom_style atomic

lattice fcc 1.073
region box block 0 10 0 10 0 10
create_box 1 box
create_atoms 1 box
mass 1 1.0

pair_style lj/cut 5.0
pair_coeff 1 1 1.0 1.0 5.0
velocity all create 0.01 87287

timestep 0.005

# Equilibrium step
fix 1 all nvt temp 0.85 0.85 0.2
run 50000
unfix 1

# Heating step
reset_timestep 0
thermo 1000
thermo_style custom step temp pe etotal press vol
fix 1 all npt temp 0.85 0.01 1.0 iso 0 0 1.0
dump 1 all atom 10000 quench.lammpstrj
run 1000000

*Solidification process demonstration*：

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/da67be899d33469ba0405ab9411d9c77/c1384d7c-7584-4c29-82af-b1b4d8b242f1.gif)

*V-T diagram*：

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/da67be899d33469ba0405ab9411d9c77/62a7052e-98d9-4718-9f90-2479164d26ea.png)


The calculation results show that the molecular dynamics simulation of the solidification process will produce **supercooling** phenomenon.



>  **Improvement**: The melting or solidification process is performed under the **NPT ensemble**, maintaining the temperature at a specific temperature for a period of time to achieve thermodynamic equilibrium, which can partially suppress overheating or undercooling.
> 
> 
> 
> **Discussion**: Molecular dynamics simulations of melting or solidification often involve *overheating* or *undercooling*.
> 
> **Nucleation** is a key concept in understanding phase transitions, including melting and solidification. Taking the melting process as an example, nucleation refers to the formation of small liquid-phase clusters within a solid. These clusters act as "seeds" for growth, ultimately leading to complete melting of the solid. *Nucleation must overcome an energy barrier.*
> 
> - **Critical size**: There is a critical size above which nuclei are energetically favorable to grow, while nuclei smaller than this size shrink and disappear. *The formation of nuclei larger than the critical size is a low probability event at the spatial scale of typical molecular dynamics simulations.*
> 
> - **Thermal Fluctuations**: Nuclear fission is often caused by thermal fluctuations, which occasionally concentrate enough energy in a small area to overcome the nuclear fission barrier. *The occurrence of thermal fluctuations within the time scale of a typical molecular dynamics simulation is a rare event.*
> 
> - Nucleation Sites: In real materials, impurities, defects, or surfaces often act as nucleation sites that lower the barrier to nucleation. *These are not usually present in the idealized conditions of simulations.*
> 
> **Conclusion: Nucleation is more difficult at the initial stage of phase transition at the time and space scales involved in molecular dynamics simulations.**



#### 3.4.3 Melting point of bulk material

##### 3.4.3.1 Input File

In [ ]:
%%writefile in.Cu_fusion

units metal
boundary p p p
atom_style atomic

lattice fcc 3.62
region box block 0 8 0 8 0 8
create_box 1 box
create_atoms 1 box

pair_style eam
pair_coeff 1 1 Cu_u3.eam

thermo 1000
thermo_style custom step temp pe etotal press vol

velocity all create 100.0 4928459 dist gaussian
fix 1 all npt temp 100.0 100.0 0.1 iso 0.0 0.0 1.0
run 10000
unfix 1

reset_timestep 0
compute 1 all msd
fix msd all ave/time 10 100 1000 c_1[1] c_1[2] c_1[3] c_1[4] file msd.dat     # Output msd
fix 1 all npt temp 100 2000 0.1 iso 0 0 1.0
dump 1 all atom 1000 dump.lammpstrj
run 120000

##### 3.4.3.2 Melting point calculation

**Mean Square Displacement (MSD) indicates the degree to which atoms deviate from their equilibrium positions**:

$$
MSD=\langle|r(t)-r(0)|^2\rangle 
$$

The mean square displacement (MSD) and the atomic diffusion coefficient have a corresponding relationship:

- The mean square displacement of solid materials has an upper limit;

- The mean square displacement of liquids has a linear relationship with temperature, and its slope is related to the atomic diffusion coefficient as follows:
  
  $$
  {D}=\lim_{t\to\infty}\frac1{2\sigma t}\langle|\boldsymbol{r}(t)-\boldsymbol{r}(0)|^2\rangle 
  $$
  
  where $\sigma$ is the diffusion dimension.

#### 3.4.4 Nanomaterials

##### 3.4.4.1 Nanoparticles

Compared with bulk structures, nanoparticles have higher surface energy, more specific surface atoms, incomplete coordination of surface atoms, and greater activity than bulk materials. Therefore, the increase in internal energy required for melting is much smaller and the melting point is lower.

- *Cu-Ni nanoparticles*：

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/ac1147c726b649a293c2360a1772ee85/03dffa66-9bc1-4f56-9d65-82bf731f1850.png)

- *Pt nanoparticles*：

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/811137b4d268442ca4adda97be726b09/72731eaf-9872-47ea-84dd-96e78baee84b.png)

Size Effect of Melting Point:

The phenomenon of melting point decreasing with decreasing size is often referred to as **melting point depression**. This decrease in melting point is primarily due to the fact that the smaller the nanoparticle, the greater the proportion of surface atoms.

$$
T_m(r)=T_m(\infty)\left(1-\alpha\frac dD\right)\\=T_m(\infty)-C/r
$$



Related physical models:

- Liquid Drop Model

- Liquid Shell Nucleation Model

- Surface Phonon Instability Model

- Bond Order Length Strength Model



**Sintering**: The process by which atoms within particles fuse across their boundaries to form a single unit. Sintering is a common molding process for some high-melting-point materials. Metal nanoparticles are often used as catalysts, deriving their superior catalytic performance from their large surface area. Sintering reduces the surface area and alters the surface structure of the catalyst, significantly contributing to the loss of catalytic activity. Therefore, anti-sintering is a critical issue in catalysts that needs to be addressed urgently.





*Ref*: 

[*Nat Commun* **10**, 2583 (2019)](https://doi.org/10.1038/s41467-019-10713-z)

[*Sci Rep* **11**, 19297 (2021)](https://doi.org/10.1038/s41598-021-98704-3)

[*Materials* **2020**, *13*(7), 1507](https://doi.org/10.3390/ma13071507)



##### 3.4.4.2 Lindemann Index

The [Lindemann index](https://en.wikipedia.org/wiki/Lindemann_index) is a simple measure of thermally driven disorder in atoms or molecules, commonly used to calculate the melting point of non-periodic systems, such as nanoparticles. It is defined as the relative root mean square of bond length fluctuations, $\delta$:

$$
q_i=\frac{1}{N-1}\sum_{j\neq i}\frac{\sqrt{\langle r_{ij}^2\rangle-\langle r_{ij}\rangle^2}}{\langle r_{ij}\rangle}
$$

or

$$
\delta=\frac2{N(N-1)}\sum_{i<j}\frac{\sqrt{\left\langle r_{ij}^2\right\rangle_t-\left\langle r_{ij}\right\rangle_t^2}}{\left\langle r_{ij}\right\rangle_t}
$$



*Ref*:

[*J. Phys. Chem. A* 2006, 110, 4, 1518–1523](https://doi.org/10.1021/jp053318s "DOI URL")

In [ ]:
from ase import io
import numpy as np


def lindemann(filename, n_start=None, n_end=None):
    """
    Calculate the Lindemann index for atomic vibrations using the provided trajectory.

    Parameters:
    - filename: Name of the file containing atomic positions (typically a LAMMPS dump file).
    - n_start: Starting index for the trajectory.
    - n_end: Ending index for the trajectory.

    Returns:
    - Lindemann index value.
    """

    # Construct the slice string based on provided indices
    if n_start is None and n_end is None:
        slice_str = ":"
    elif n_start is None:
        slice_str = f":{n_end}"
    elif n_end is None:
        slice_str = f"{n_start}:"
    else:
        slice_str = f"{n_start}:{n_end}"

    # Read atomic positions from the file for the given range
    atoms = io.read(filename, slice_str, 'lammps-dump-text')

    # Total number of atoms and total number of timesteps
    N = atoms[0].positions.shape[0]
    N_t = len(atoms)
    print('number of atoms in each frame =', N, 'number of frames =', N_t)

    # Initialize arrays to store distances between atoms
    r_square = np.zeros((N_t, N, N))

    # Loop through each timestep
    for t in range(N_t):
        G = np.dot(atoms[t].positions, atoms[t].positions.T)
        H = np.tile(np.diag(G), (N, 1))

        # Compute the squared distance between each pair of atoms at time t
        r_square[t, :, :] = H + H.T - 2 * G

    # Compute average distance and squared average distance over all timesteps
    r_ave = np.mean(np.sqrt(r_square), axis=0)
    print(r_square.shape)
    print(r_ave.shape)
    print('r_ave_max =', np.max(r_ave), 'r_ave_min =', np.min(r_ave))

    r2_ave = np.mean(r_square, axis=0)
    print('r2_ave =', np.max(r2_ave))

    # Extract upper triangular part of matrices
    r_ave_upper = r_ave[np.triu_indices(N, k=1)]
    print('r_ave_upper_max =', np.max(r_ave_upper), 'r_ave_upper_min =', np.min(r_ave_upper))
    r2_ave_upper = r2_ave[np.triu_indices(N, k=1)]

    # Calculate Lindemann criterion for upper triangular elements
    value_upper = np.sqrt(r2_ave_upper - np.power(r_ave_upper, 2)) / r_ave_upper

    # Sum over all unique atom pairs
    total_value = np.sum(value_upper)

    # Normalize by the total number of atom pairs
    return total_value * 2 / (N * (N - 1))

##### 3.4.4.3 Input File

- Nanoparticle melting process

In [ ]:
%%writefile in.nanoparticles_fusion

# Initialization
units metal
boundary p p p
atom_style atomic

# Generate NiCu nanoparticles
variable A0 equal 3.589
lattice fcc ${A0}
region mybox block 0 40 0 40 0 40
create_box 2 mybox                         # 2 types of atoms
region CuNi_nano sphere 20 20 20 2
create_atoms 1 region CuNi_nano
set type 1 type/fraction 2 0.5 7777
mass 1 63.54600000                         # Cu
mass 2 58.69340000                         # Ni

# Using EAM potential
pair_style eam/alloy
pair_coeff * * CuNi.eam.alloy Cu Ni
write_data nanoparticle.cif
run 0

# Thermal equilibrium step
thermo 1000
variable j loop 0 20
label loop_j
variable temperature equal 900+10*$j
variable T equal temp
variable Eatom equal etotal/atoms
fix 1 all nvt temp ${temperature} ${temperature} 0.1
run 50000
unfix 1

# Statistical step
fix 2 all ave/time 100 5 1000 v_T v_Eatom file data_ave${temperature}.txt
dump 1 all atom 5000 fusion_${temperature}.atom
fix 1 all nvt temp ${temperature} ${temperature} 0.1
run 500000

unfix 1
undump 1
next j
jump SELF loop_j

*Demonstration of the nanoparticle melting process*：

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/ac1147c726b649a293c2360a1772ee85/2b01860b-1d1b-4289-ae20-a8b55ec47013.gif)

*Lindemann index - T diagram*：

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/ac1147c726b649a293c2360a1772ee85/3179fdd3-dbc4-461d-88ad-686aa040d127.png)


- Nanoparticle sintering process

In [ ]:
%%writefile in.nanoparticles_sintering

# Initialization
units metal
boundary p p p
atom_style atomic
timestep 0.001

# Get three Pt nanoparticles 
variable A0 equal 3.9239
lattice fcc ${A0}
region mybox block 0 20 0 20 0 20
region sphere_pt1 sphere 7 10 8 3
region sphere_pt2 sphere 10 10 13.4 3
region sphere_pt3 sphere 13 10 8 3
create_box 1 mybox
create_atoms 1 region sphere_pt1
create_atoms 1 region sphere_pt2
create_atoms 1 region sphere_pt3

# Using EAM potential
pair_style eam
pair_coeff 1 1 Pt_u3.eam

# Output initial structure
dump 1 all cfg 1 coord.*.cfg mass type xs ys zs
run 0
undump 1

# Define triple_neck region
region triple_neck block 7.5 12.5 5 15 9 13
group 1 dynamic all region triple_neck every 1000

# Output number of atoms in triple neck region
variable N equal step
variable T equal temp
variable Natom equal count(all)
variable V equal vol/v_Natom
variable sinter_atom equal count(1)
dump 1 all xyz 1000 melt.xyz
thermo 1000
fix extra all print 1000 "${N} ${T} ${sinter_atom}" file data.txt

# Run in 500k, 1000k, 1400k, respectively
fix 1 all npt temp 500 500 0.1 iso 1 1 1
run 10000
unfix 1
fix 1 all npt temp 1000 1000 0.1 iso 1 1 1
run 20000
unfix 1
fix 1 all npt temp 1400 1400 0.1 iso 1 1 1
run 30000
unfix 1

*Demonstration of the nanoparticle sintering process*：

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/811137b4d268442ca4adda97be726b09/2ae29c08-489b-43ef-abce-28d1bb580d63.gif)

*Temp/Atom_number - Steps diagram*：

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/811137b4d268442ca4adda97be726b09/d134ebfa-ede2-406a-98af-cb7ca87b55c2.png)

## 4 Transport properties

### 4.1 Ion transport (energy materials)

By simulating the motion of atoms, molecules, and other particles under their interaction forces, we can explore the structure and dynamic properties of materials. This approach is widely applicable to studying the atomic-scale interaction mechanisms of energy materials, including binding energy patterns, deformation mechanisms, heat transport patterns, segregation phase formation, and other behaviors closely related to energy materials. It is of great significance in energy materials research:

- Material phase formation and property prediction: such as phase transitions, mechanical strength, thermal conductivity, and volume expansion.

- Interface and surface research: such as in solar cells, energy storage devices, and electrochemical cells.

- Ion transport and diffusion: such as in lithium-ion batteries, fuel cells, and electrolyte materials.

- Hydrogen storage material research: It can simulate the adsorption, diffusion, and release processes of hydrogen in hydrogen storage materials.

#### 4.1.1 Li-S battery expansion

**Basic components of lithium-ion battery electrodes**:

- Anode (negative electrode)

  During discharge, it loses electrons from the external circuit; an oxidation reaction occurs; its potential is low.

- Separator

- Cathode

  During discharge, it gains electrons from the external circuit; a reduction reaction occurs; its potential is high.

- Organic electrolyte

- Battery casing

Lithium batteries are prone to swelling during high-temperature storage and cycling, which can lead to a series of safety issues:

1. Expansion of the electrode material can cause physical damage to battery components;

2. Volume expansion can lead to physical contact between the positive and negative electrodes, causing a battery short circuit;

3. It can cause electrode material failure, reducing the battery's capacity and cycle life.

**LiS Batteries**

- Advantages

  - High specific capacity

  - High energy density

  - Low cost

  - Environmentally friendly

- Disadvantages

  - Low conductivity and low discharge efficiency

  - Polysulfide ions dissolve in the electrolyte, resulting in poor cycling performance

  - Large volume expansion and short cycle life

*Ref*:

- [*Acc. Chem. Res.* 2013, 46, 5, 1135–1143](https://pubs.acs.org/doi/10.1021/ar3001348)

- [*J. Phys. Chem. Lett.* 2013, 4, 19, 3227–3232](https://pubs.acs.org/doi/10.1021/jz401763d)

##### 4.1.1.1 Simulation Outline

1. Randomly generate a certain number of $Li$ atomic coordinates and fill them into the $S$ structure as the $$Li_xS_{128}$ initial structure;

2. After $Li$ is embedded, $S$ becomes an amorphous phase. Consider **heating melting + rapid annealing** to construct an amorphous structure;

3. The volume of the $Li_xS$ amorphous phase at room temperature was calculated in the *NPT* ensemble and standard atmospheric pressure, and the volume expansion rate was calculated by comparing it with the volume before $Li$ embedding.

**Potential function**：[***Phys. Chem. Chem. Phys.***, 2015,**17**, 3383-3393](https://pubs.rsc.org/en/content/articlelanding/2015/cp/c4cp04532g)

##### 4.1.1.2 Input File

In [ ]:
%%writefile in.thermal_expansion

units real
boundary p p p
atom_style charge

read_data Li2S.dat

pair_style reax/c NULL checkqeq no
pair_coeff * * ffield.reax.LiS S Li

# Fix 1 all qeq/reax 1 0.0 10.0 1e-6 reax/c
neighbor 3.0 bin
min_style cg
minimize 1e-4 1e-6 100 1000

# Stablize at room Temp
velocity all create 300 96588
thermo 1000
thermo_style custom step temp press density vol
compute myRDF all rdf 50 1 1 1 2 2 1 2 2
fix 2 all ave/time 100 10 10000 c_myRDF[*] file initial.rdf mode vector
fix 3 all nvt temp 300 300 10
run 10000
unfix 2
unfix 3

# Heating slowly
fix 4 all npt temp 300.0 1800.0 100 iso 1.0 1.0 1000 drag 1.0
run 50000
unfix 4

# Keep melting
fix 5 all npt temp 1800.0 1800.0 100 iso 1.0 1.0 1000 drag 1.0
fix 6 all ave/time 100 10 5000 c_myRDF[*] file melted.rdf mode vector
run 5000
unfix 5
unfix 6

# Rapidly quenching
fix 7 all npt temp 1800.0 300.0 1 iso 1.0 1.0 1000 drag 0.1
thermo 10
run 100
unfix 7

# Stablize the amorphous LixS at RT
fix 8 all ave/time 100 10 5000 c_myRDF[*] file amorphous.rdf mode vector
fix 9 all npt temp 300 300 100 iso 1.0 1.0 1000 drag 1.0
thermo 1000


> [Radial Distribution Function (RDF)](https://en.wikipedia.org/wiki/Radial_distribution_function)
> 
> ![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/c7aa6c89e33141a384510c426c263a08/cf983c01-462b-4a03-afcd-11b5d2a054df.png)
> 
> 
> Relative atomic density as a function of radius
> 
> $$
g(r)=\frac{\left\langle N(r\pm\frac{\Delta r}2)\right\rangle}{\Omega(r\pm\frac{\Delta r}2)}\cdot\frac1\rho 
$$
> 
> This physical quantity can be used to characterize the **local structural order** of atoms and identify the change from order to disorder.
> 
> - Solids: neighboring atoms appear at characteristic distances;
> 
> - Liquid: The distance between neighboring atoms changes smoothly and continuously near the characteristic distance;
> 
> - Gas: There is no long-range and short-range order among neighboring atoms.

#### 4.1.2 Hydrogen storage mechanism of Mg

**Current Status of Hydrogen Storage**:

- High-pressure cylinders: High density, high pressure, and safety risks;

- Porous adsorption: Low specific gravity, weak adsorption;

- Compound hydrogen: High specific gravity, strong adsorption, and difficult to desorb and release.

##### 4.1.2.1 Simulation Outline

Research object: Mg(100) surface

Research objectives:

1. To investigate the hydrogen storage mechanism of Mg;

2. To investigate the effect of different temperatures on the hydrogen storage effect of Mg.

**Potential function**：[***Computational Materials Science*** 154 (2018) 295–302](https://www.sciencedirect.com/science/article/pii/S0927025618304865)

##### 4.1.2.2 Input File

In [ ]:
%%writefile in.hydrogen_storage

units metal
dimension 3
atom_style atomic
boundary p p p
read_data MgH.data
neighbor 0.5 bin
neigh_modify every 1 delay 0 check yes

region Mg block INF INF INF INF -0.01 5 units box
group Mg region Mg
velocity Mg set 0 0 0
fix 01 Mg setforce 0 0 0
region H2 block INF INF INF INF 6 INF units box
group H2 region H2

mass 1 24.3050
mass 2 1

pair_style adp
pair_coeff * * Mg_H.adp.alloy.txt Mg H

region 1H block INF INF INF 9 INF units box
group mobile1 dynamic all region 1H every 100
variable number1 equal count(mobile1)

region 2H block INF INF INF 5 9 units box
group mobile2 dynamic all region 2H every 100
variable number2 equal count(mobile2)

region 3H block INF INF INF -0.001 5 units box
group mobile3 dynamic all region 3H every 100
variable number3 equal count(mobile3)

# compute 1 Mg temp
compute 2 H2 temp
timestep 0.001

variable T equal TEMP
thermo 1000
thermo_style custom step temp pe etotal press vol c_2

min_style cg
minimize 1e-5 1e-5 10000 10000
run 0

velocity H2 create ${T} 71717
dump 1 all atom 100 dump.xyz
fix 1 H2 nvt temp ${T} ${T} 0.1
thermo_modify lost ignore
fix extra all print 100 "${N} ${P} ${number1} ${number2} ${number3}" file energy.data
run 200000

### 4.2 Heat transport (thermoelectric materials)

#### 4.2.1 Theoretical basis of heat conduction

Heat Transfer Mechanisms:

- Solids: Heat conduction

- Liquids: Heat convection

- Gases: Convection and radiation

Heat conduction: The process by which heat is transferred from a high-temperature portion of a solid material to a low-temperature portion.

According to the Second Law of Thermodynamics (stated by Clausius), under natural conditions, heat can only be transferred from a high-temperature object to a low-temperature object. This transfer process is **spontaneous** and **irreversible**.

According to the Third Law of Thermodynamics, atoms are constantly vibrating around their lattice equilibrium positions. **The physical nature of a material's various thermal properties is related to the thermal vibrations of its crystal lattice.**

Thermal conduction in insulating materials is primarily caused by atomic (lattice) vibrations.

In quantum mechanics:

$$
H=\frac12m\dot{x}^2+\frac12kx^2=\frac12\dot{q}^2+\frac12\omega^2q^2
$$

A harmonic oscillator has discrete eigenenergies:

$$
\varepsilon_n=\left(n+\frac12\right)\hbar\omega,\quad n=0,1,\cdots,\infty 
$$

A [Phonon](https://en.wikipedia.org/wiki/Phonon) is defined as the energy quantum of lattice vibration. The phonon energy $\propto\hbar\omega=\hbar\sqrt{\frac km}$. Collisions between phonons result in heat transfer. Therefore, the stronger the intermolecular forces (bonds), the greater the thermal conductivity.

Phonon collision process:

- Normal process: No thermal resistance

$$
\vec{q}_1+\vec{q}_2=\vec{q}_3
$$

- Umklap process: With thermal resistance

$$
\vec{q}_1+\vec{q}_2=\vec{q}_3+\vec{K}_h
$$

> - Long-wave phonons have longer wavelengths and are more prone to inversion processes, so they have a greater impact on thermal conductivity properties.
> 
> - The contribution of acoustic phonons to thermal conductivity is much greater than that of optical phonons.



#### 4.2.2 Thermal conductivity

[Thermal conductivity](https://en.wikipedia.org/wiki/Thermal_conductivity_and_resistivity) is a measure of a material's ability to **conduct heat**.

Standard unit: $W/(m\cdot K)$

$$
\kappa=\frac13C\bar{v}\bar{\lambda}
$$

- Proportional to the **specific heat** $C$ of the solid. The greater the specific heat, the more heat each solid molecule can carry.

- Proportional to the **average phonon velocity** $\bar{v}$. The faster the phonon velocity, the faster the heat transfer.

- Proportional to the **mean free path** $\bar{\lambda}$ (related to the phonon collision frequency). The larger $\bar{\lambda}$, the farther the phonons carry heat.

In the above formula, the average phonon velocity is not significantly correlated with temperature changes and can be treated as a constant in most cases. The mean free path is more difficult to calculate and depends on factors such as temperature and the type and frequency of phonon collisions.

**Crystal heat capacity**:

$$
C_V=\left(\frac{\partial U}{\partial T}\right)_V
$$

The heat capacity of solids comes primarily from two sources:

- Lattice vibrations (phonon motion) – lattice heat capacity $C_V^l$

- Electron thermal motion – electron heat capacity $C_V^e$

$$
C_V=C_V^l+C_V^e
$$

Typically, $C_V^e<<C_V^l$. At very low temperatures, the contribution of the electron heat capacity must be considered.

***Experimental Rule***:

1. At high temperatures, the crystal heat capacity
   
   $$
   C_V=3Nk_B=3\nu R=24.9J/K\cdot mol
   $$

2. At low temperatures,

   - The heat capacity of an insulator is $\propto T^3$;

   - The heat capacity of a conductor is $\propto T$.


The energy of lattice vibrations is quantized, and the energy of a phonon with a frequency of $\omega$ is

$$
E_n=(n+\frac12)\hbar\omega
$$

where $\frac12\hbar\omega$ represents the zero-point vibrational energy, which does not contribute to the heat capacity, so

$$
E_n=n\hbar\omega
$$

where $n$ is the average number of phonons in a harmonic oscillator with a frequency of $\omega$. According to **Bose statistics**,

$$
n(\omega)=\frac1{e^{\hbar\omega/k_BT}-1}
$$

Therefore, at temperature $T$, the vibration energy of frequency $\omega$ is

$$
\bar{E}(\omega)=\frac{\hbar\omega}{e^{\hbar\omega/k_BT}-1}
$$

The crystal is composed of $N$ atoms, each atom has $3$ degrees of freedom, and there are a total of $3N$ discrete vibration frequencies. The internal energy of the crystal is

$$
U=\sum_{i=1}^{3N}\bar{E}\left(\omega_i\right)=\sum_{i=1}^{3N}\frac{\hbar\omega_i}{e^{\hbar\omega_i/k_BT}-1}
$$

- **Einstein model** assumes that all atoms vibrate at the same frequency,

$$
U=\sum_{i=1}^{3N}\bar{E}\left(\omega_i\right)=\frac{3N\hbar\omega}{e^{\hbar\omega/k_BT}-1}
$$

This model's specific heat capacity is $3Nk_B$ at high temperatures, consistent with classical results, but deviates significantly at low temperatures.

- **Debye model** represents the frequency distribution using an integral function.

$$
\int_0^{\omega_m}g(\omega)d\omega=3N
$$

Thus,

$$
U=\sum_{i=1}^{3N}\frac{\hbar\omega_i}{e^{\hbar\omega_i/k_BT}-1} =\int_0^{\omega_m}\frac{\hbar\omega}{e^{\hbar\omega/k_bT}-1}g(\omega)d\omega 
$$

Heat capacity

$$
C_V=\left(\frac{\partial U}{\partial T}\right)_V=\int_0^{\omega_m}k_B\left(\frac{\hbar\omega}{k_BT}\right)^2\frac{e^{\hbar\omega/k_BT}g(\omega)d\omega}{(e^{\hbar\omega/k_BT}-1)^2}
$$

After variable substitution

$$
f_D\left(\frac{\theta_D}T\right)=3\left(\frac T{\theta_D}\right)^3\int_0^{\frac{\theta_D}T}\frac{e^xx^4}{(e^x-1)^2}dx
$$

Then at low temperatures, $C_V{\sim}\frac{12\pi^4R}5{(\frac T{\theta_D})}^3$

The **phonon mean free path** is inversely proportional to the phonon number density.

$$
\bar{\lambda}\propto\frac1n\propto e^{\frac\alpha T}
$$

- At low temperatures, the phonon mean free path $L$ is approximately equal to the particle size, $\kappa\propto T^3$;

- As temperature increases, the heat capacity increases, while the phonon mean free path decreases. Competition between the two leads to a maximum value for $\kappa$;

- At high temperatures, the phonon mean free path $L$ decreases with increasing temperature, the phonon heat capacity $C_V$ approaches a constant, and $\kappa$ decreases.

**Thermoelectric Materials**

- [**Thermoelectric effect**](https://en.wikipedia.org/wiki/Thermoelectric_effect)
  
  - [**Seebeck effect**](https://en.wikipedia.org/wiki/Thermoelectric_effect#Seebeck_effect)
  
  - [**Peltier effect**](https://en.wikipedia.org/wiki/Thermoelectric_effect#Peltier_effect)

- Thermoelectric figure of merit

$$
ZT=\frac{S^2T\sigma}K
$$

[***Fourier's law***](https://en.wikipedia.org/wiki/Thermal_conduction#Fourier's_law)

> Analogy to [Ohm's law](https://en.wikipedia.org/wiki/Ohm%27s_law)
> 
> **Ohm's law** describes the transport of electrons within a conductive material:
> 
> $$
\frac{dq}{dt}=\sigma\cdot A\frac Vl
$$
> 
> **Fourier's law** describes the transport of heat within a material:
> 
> $$
\frac{dQ}{dt}=\kappa\cdot A\frac{\Delta T}l
$$



##### 4.2.2.1 Calculation Outline

Fourier's law:

$$
\frac{dQ}{dt}=\kappa\cdot A\frac{\Delta T}l
$$

Calculate the heat flux and temperature gradient per unit cross-sectional area to determine the thermal conductivity of the material.

- Apply heat flux along a specific length of the material using a specific method and calculate the thermal power.

- After a certain period of time, the internal temperature of the material reaches a steady state and the temperature gradient is calculated.

- Calculate the thermal conductivity by combining the cross-sectional area and length parameters.

##### 4.2.2.2 Input File

- ***Fixed Heat Flux Exchange Method***


In [ ]:
%%writefile in.langevin

units lj
atom_style atomic
lattice fcc 0.6
region box block 0 10 0 10 0 20
create_box 1 box
create_atoms 1 box
mass 1 1.0
velocity all create 1.35 71717
pair_style lj/cut 2.5
pair_coeff 1 1 1.0 1.0
neighbor 0.3 bin

region hot block INF INF INF INF 0 1
region cold block INF INF INF INF 10 11

compute Thot all temp/region hot
compute Tcold all temp/region cold

# 1st Equilibration run
fix 1 all nvt temp 1.35 1.35 0.5
thermo 100
run 1000

velocity all scale 1.35
unfix 1

# 2nd Equilibration run
fix 1 all nve
fix hot all heat 1 100.0 region hot
fix cold all heat 1 -100.0 region cold
thermo_style custom step temp c_Thot c_Tcold
thermo 1000
run 10000

compute ke all ke/atom
variable temp atom c_ke/1.5

compute layers all chunk/atom bin/1d z lower 0.05 units reduced
fix 2 all ave/chunk 10 100 1000 layers v_temp file profile.heat
variable tdiff equal f_2[11][3]-f_2[1][3]

fix ave all ave/time 1 1 1000 v_tdiff ave running start 13000
thermo_style custom step temp c_Thot c_Tcold v_tdiff f_ave
run 20000

- ***Muller-Plathe method (velocity exchange)***

In [ ]:
%%writefile in.mp

units lj
atom_style atomic
lattice fcc 0.6
region box block 0 10 0 10 0 20
create_box 1 box
create_atoms 1 box
mass 1 1.0
velocity all create 1.35 71717
pair_style lj/cut 2.5
pair_coeff 1 1 1.0 1.0
neighbor 0.3 bin

# 1st Equilibration run
fix 1 all nvt temp 1.35 1.35 0.5
thermo 100
run 1000
velocity all scale 1.35
unfix 1

# 2nd Equilibration run
compute ke all ke/atom
variable temp atom c_ke/1.5
fix 1 all nve
compute layers all chunk/atom bin/1d z lower 0.05 units reduced
fix 2 all ave/chunk 10 100 1000 layers v_temp file profile.mp
fix 3 all thermal/conductivity 10 z 20
variable tdiff equal f_2[11][3]-f_2[1][3]
thermo_style custom step temp epair etotal f_3 v_tdiff
thermo 1000
run 20000

# Thermal conductivity calculation
fix 3 all thermal/conductivity 10 z 20
fix ave all ave/time 1 1 1000 v_tdiff ave running
thermo_style custom step temp epair etotal f_3 v_tdiff f_ave
run 20000

##### 4.2.2.3 Notes

- The key and difficulty in calculating thermal conductivity using Fourier's law is **generating a stable, linear temperature gradient distribution in the system**;

- If a temperature gradient cannot be obtained, **adjust parameters such as the ensemble, heat flux exchange rate, and time step**;

- The temperature gradient should not be too large. Generally, **the temperature difference between the high and low temperature zones should be kept within 100 K**, and the average value should be close to the given ensemble temperature, with minimal drift.

#### 4.2.3 Summary

##### 4.2.3.1 Non-equilibrium Molecular Dynamics (NEMD)

Methods for calculating thermal conductivity using Fourier's law:

- During the simulation, the energy of each component of the object is in a non-equilibrium state.

- The fixed heat flux exchange method involves artificial energy exchange with the outside world.

- The MP (velocity exchange) method involves artificial energy exchange between groups of atoms.

- After a period of time, when the diffusion efficiency is equivalent to the artificial energy exchange, the temperature of each group in the system tends to stabilize.

- Methods for calculating thermal conductivity based on Fourier's law are often referred to as non-equilibrium molecular dynamics simulations (NEMD).

In addition, the thermal conductivity of anisotropic materials is a tensor. NEMD can only obtain thermal conductivity in one direction at a time, making it suitable for simulating the thermal conductivity of low-dimensional materials.



##### 4.2.3.2 Equilibrium Molecular Dynamics (EMD)

Based on the [Green-Kubo linear response theory](https://en.wikipedia.org/wiki/Green%E2%80%93Kubo_relations), the thermal conductivity of a material is related to the fluctuation-dissipation heat flow in the equilibrium state. Thermal conductivity can be calculated using the equilibrium heat flow autocorrelation function.

$$
\kappa_{\mu\nu}(\tau_m)=\frac1{\Omega_{k_B}T^2}\int_0^{\tau_m}\left\langle\bar{J}_\mu(\tau)J_\nu(0)\right\rangle d\tau 
$$

Discretize the time steps and average them,

$$
\kappa_{\mu v}(\tau_M)=\frac{\Delta t}{\Omega kBT^2}\sum_{m=1}^M(N-m)^{-1}\sum_{m=1}^{N-m}J_\mu(m+n)J_\nu(n)
$$

This is the formula for calculating thermal conductivity using the molecular dynamics method in equilibrium.



*ref*: [Phys. Rev. B **65**, 144306](https://doi.org/10.1103/PhysRevB.65.144306)